In [51]:
# import netgen.gui
import ngsolve as ngs
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw
from ttictoc import tic, toc
from ngsolve.krylovspace import CGSolver

mesh = ngs.Mesh('team10_2d.vol')
mesh.Curve(2)
# mesh.Refine()

In [52]:
%run J_2D.ipynb

using 3d mesh with ne= 178544  elements and nv= 89873  points


In [53]:
from bhdata import BHCurves
fun_dw  = BHCurves(4)
fun_w   = fun_dw.Integrate()
fun_ddw = fun_dw.Differentiate()

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil|ambient|default"
nonlinear = "r_steel|l_steel|mid_steel"

lin = 0

BH data from TEAM 13 problem
returning energy


In [54]:


Q = ngs.VectorL2(mesh, order = deg-1)
V = ngs.H1(mesh, order = deg)
# V = ngs.H1(mesh, order = p, dirichlet = "ambient_face")

X = Q*V
(p,u), (q,v) = X.TnT()

# Nonlinear:

maxit = int(1e7)

tol2 = 1e-8
regb = 1e-14

B_psi = ngs.GridFunction(X)

B = B_psi.components[0]
psi = B_psi.components[1]

normB = ngs.sqrt(B*B + regb)

if lin == 1: cf_energy = mesh.MaterialCF({linear: nu0/2*B*B, nonlinear: nu0/2*B*B}, default = nu0/2*B*B).Compile()
cf_energy = mesh.MaterialCF({linear: nu0/2*B*B, nonlinear: fun_w(normB)}, default = nu0/2*B*B).Compile()

def fun_W():
    with ngs.TaskManager(): res = ngs.Integrate(cf_energy - Hs*B, mesh)
    return res

if lin == 1: cf_rhs = mesh.MaterialCF({linear: nu0, nonlinear: nu0}, default = nu0).Compile()
else: cf_rhs = mesh.MaterialCF({linear: nu0, nonlinear: fun_dw(normB)/normB}, default = nu0).Compile()

ir = ngs.IntegrationRule(points = [(1/3,1/3)], weights = [1/2])

rhs = ngs.LinearForm((cf_rhs*B*q -ngs.grad(psi)*q -Hs*q +B*ngs.grad(v))*ngs.dx)
# rhs = ngs.LinearForm(X)
# rhs += ngs.SymbolicLFI(cf_rhs*B*q -ngs.grad(psi)*q -Hs*q +B*ngs.grad(v), intrule = ir)

def fun_dW(): #implicitly depending on A!
    with ngs.TaskManager(): rhs.Assemble()
    return rhs

Id = ngs.CF((1,0,
             0,1), dims=(2,2)).Compile()

BBt = ngs.CF((B[0]*B[0], B[0]*B[1],
              B[1]*B[0], B[1]*B[1]), dims = (2,2)).Compile()

fun1 = fun_dw(normB)/normB
fun2 = (fun_ddw(normB) - fun_dw(normB)/normB)/(normB*normB)

if lin == 1: cf_iter = mesh.MaterialCF({linear: nu0*Id, nonlinear: nu0*Id}, default = nu0*Id).Compile()
else: cf_iter = mesh.MaterialCF({linear: nu0*Id, nonlinear: fun1*Id + fun2*BBt}, default = nu0*Id).Compile()

K_iter = ngs.BilinearForm(X)
K_iter += ngs.SymbolicBFI(cf_iter*p*q - ngs.grad(u)*q + p*ngs.grad(v))
# K_iter += ngs.SymbolicBFI(cf_iter*p*q - ngs.grad(u)*q + p*ngs.grad(v), intrule = ir)
C_iter = ngs.Preconditioner(K_iter, type = "local")

def fun_ddW():
    with ngs.TaskManager(): K_iter.Assemble()
    return K_iter


In [55]:
print("Using 3D mesh with ne=", mesh.ne, "elements and nv=", mesh.nv, "points and " ,X.ndof, "DOFs.\n ")

with ngs.TaskManager(): B.Set(ngs.CF((0,0)))
with ngs.TaskManager(): psi.Set(ngs.CF((0)))

du = ngs.GridFunction(X)
uo = ngs.GridFunction(X)
wo = 1e12

for it in range(1,maxit+1):
    
    tic()
    # with ngs.TaskManager():
    #     K_iter.Assemble()
    #     rhs.Assemble()
    #     res = ngs.Integrate(cf_energy - Hs*ngs.curl(A), mesh)
    
    w  = fun_W()
    dw = fun_dW()
    da = fun_ddW()
    tm1 = toc()
    
    tic()
    with ngs.TaskManager():
        du.vec.data = da.mat.Inverse(X.FreeDofs()) * dw.vec
        # ngs.solvers.MinRes(mat=da.mat, pre=C_iter.mat, rhs=dw.vec, sol=du.vec, printrates=False, initialize=False, maxsteps=10000)
        # iterativeSolver = CGSolver(K_iter.mat, freedofs = X.FreeDofs(), atol = 1e-2,  maxiter = maxit, printrates = False)
        iterativeSolver = CGSolver(K_iter.mat, pre = C_iter.mat, tol  = 1e-2,  maxiter = maxit)
        # du.vec.data = iterativeSolver * dw.vec
    
    # if len(iterativeSolver.residuals) == maxit: print("... reached maxit!")
    # print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")
    tm2 = toc()

    nrm = ngs.InnerProduct(du.vec,dw.vec)
    
    if it == 1:
        nrm0 = nrm

    wn = 1e12
    # if abs(wo-w) < tol2:
    if abs(wo-w)/abs(w) < tol2:
    # if nrm/nrm0 < tol2:
        print("converged to desired tolerance")
        break
    # elif abs(wo-w) < tol2*1e-5:
    #     print("stopped early due to stagnation")
    #     break
    else:
        # linesearch
        uo.vec.data = B_psi.vec.data
        wo = w
        alpha = 1
        for init in range(1,21):
            B_psi.vec.data -= alpha*du.vec.data
            wn = fun_W()
            if wn < w - alpha*0.01*nrm:
                print("Iter: %2d | assem : %.2fs | CG took %.2fs with %4d iterations | alpha : %.2f | energy = %.10f | relres = %.2e |"  %(it,tm1,tm2,iterativeSolver.iterations,alpha,w,nrm/nrm0))
                break
            else:
                alpha = alpha/2
                B_psi.vec.data = uo.vec.data

Using 3D mesh with ne= 178544 elements and nv= 89873 points and  2947777 DOFs.
 


NgException: PardisoInverse: Setup and Factorization failed.

In [ ]:
Draw(ngs.log(ngs.Norm(B)), mesh, min = -13, max = 0.5, settings={"Objects": {"Wireframe": False}})

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Objects': {'Wireframe': Fal…

BaseWebGuiScene

In [ ]:
print(ngs.Integrate(ngs.Norm(B),mesh, order = 0))
print(ngs.Integrate(ngs.Norm(psi),mesh))
print(fun_W())

0.0031339824702120555
10.421428788024652
-4.295485497176291


In [ ]:
scalars = mesh.MaterialCF({"coil": 1, "mid_steel": 2}, default = 0)
L2 = ngs.L2(mesh, order = 2)
scal = ngs.GridFunction(L2)
scal.Set(scalars)

vtk = ngs.VTKOutput(mesh, coefs = [B,ngs.curl(Hs),scal],names = ["B","J","scalars"], filename = "B2", order = 1)
vtk.Do()

'B2'

In [ ]:
import pyvista as pv
pvmesh = pv.read("B2.vtu")
pvmesh.set_active_scalars("scalars")
pvmesh.set_active_vectors("B")
threshed = pvmesh.threshold([0.8,2.1])
# threshed.set_active_vectors("B")

p = pv.Plotter()
p.add_mesh(threshed, style = 'surface', color = "w", opacity = 0.01, label = None)

arrows = pvmesh.glyph(scale = "B", orient = True, tolerance = 0.01, factor = 0.01)
# arrows = pvmesh.glyph(scale = "B", orient = True, tolerance = 0.01, factor = 0.01)
p.add_mesh(arrows, color = "yellow")

pvmesh.set_active_vectors("J")
arrows2 = pvmesh.glyph(scale = "J", orient = True, tolerance = 0.001, factor = 1*1e-8)
p.add_mesh(arrows2, color = "black")

p.show(jupyter_backend = "html")
# p.show()

ValueError: Data field (B) with type (FieldAssociation.POINT) could not be set as the active vectors

In [ ]:
deg = 2
print(deg)
{ir if deg==1 else None}

In [ ]:
ir.weights

In [ ]:
ngs.i